# 00016577 ML Coursework

1 . Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    mean_squared_error,
    mean_absolute_error,
    r2_score
)

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from xgboost import XGBClassifier, XGBRegressor

sns.set(style="whitegrid")
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")


2 . Data Loading

In [5]:
# 2. Load raw data
card_app = pd.read_csv("C:/Users/admin/Desktop/ML/Data/Card_application.csv")
card_credit = pd.read_csv("C:/Users/admin/Desktop/ML/Data/Card_credit.csv")




In [7]:
card_app.shape, card_credit.shape
print("Card_application:", card_app.shape)
print("Card_credit:", card_credit.shape)

display(card_app.head())
display(card_credit.head())

Card_application: (438557, 18)
Card_credit: (1048575, 3)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,"427,500.00",Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.00
1,5008805,M,Y,Y,0,"427,500.00",Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.00
2,5008806,M,Y,Y,0,"112,500.00",Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.00
3,5008808,F,N,Y,0,"270,000.00",Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.00
4,5008809,F,N,Y,0,"270,000.00",Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.00


,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


3 . Merge two Data sets

3.1 Group by and aggregations

In [9]:
# Mark statuses (bad / late )
card_credit["bad"] = card_credit["STATUS"].isin(["2", "3", "4", "5"]).astype(int)
card_credit["late"] = card_credit["STATUS"].isin(["1"]).astype(int)

credit_agg = (
    card_credit
    .groupby("ID")
    .agg(
        max_bad=("bad", "max"),                 # target for classification
        any_late=("late", "max"),
        months_on_book=("MONTHS_BALANCE", "nunique"),
        min_month=("MONTHS_BALANCE", "min"),
        max_month=("MONTHS_BALANCE", "max")
    )
    .reset_index()
)

print("Aggregated data:", credit_agg.shape)
credit_agg.head()


Aggregated data: (45985, 6)


,ID,max_bad,any_late,months_on_book,min_month,max_month
0,5001711,0,0,4,-3,0
1,5001712,0,0,19,-18,0
2,5001713,0,0,22,-21,0
3,5001714,0,0,15,-14,0
4,5001715,0,0,60,-59,0


3.2 Merging with application data

In [10]:
data = card_app.merge(credit_agg, on="ID", how="inner")
print("Merged data shape:", data.shape)
data.head()



Merged data shape: (36457, 23)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,max_bad,any_late,months_on_book,min_month,max_month
0,5008804,M,Y,Y,0,"427,500.00",Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.00,0,1,16,-15,0
1,5008805,M,Y,Y,0,"427,500.00",Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.00,0,1,15,-14,0
2,5008806,M,Y,Y,0,"112,500.00",Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.00,0,0,30,-29,0
3,5008808,F,N,Y,0,"270,000.00",Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.00,0,0,5,-4,0
4,5008809,F,N,Y,0,"270,000.00",Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.00,0,0,5,-26,-22


3.3 Applying business logic and creating target risk_score

In [12]:
data["risk_score"] = (
    1.0 * data["any_late"] +
    2.0 * data["max_bad"] +
    0.02 * data["months_on_book"]
)

data[["ID", "any_late", "max_bad", "months_on_book", "risk_score"]].head()


,ID,any_late,max_bad,months_on_book,risk_score
0,5008804,1,0,16,1.32
1,5008805,1,0,15,1.30
2,5008806,0,0,30,0.60
3,5008808,0,0,5,0.10
4,5008809,0,0,5,0.10


4 EDA (Data Analysis and Vizulaization)

In [14]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 36457 entries, 0 to 36456
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   36457 non-null  int64  
 1   CODE_GENDER          36457 non-null  object 
 2   FLAG_OWN_CAR         36457 non-null  object 
 3   FLAG_OWN_REALTY      36457 non-null  object 
 4   CNT_CHILDREN         36457 non-null  int64  
 5   AMT_INCOME_TOTAL     36457 non-null  float64
 6   NAME_INCOME_TYPE     36457 non-null  object 
 7   NAME_EDUCATION_TYPE  36457 non-null  object 
 8   NAME_FAMILY_STATUS   36457 non-null  object 
 9   NAME_HOUSING_TYPE    36457 non-null  object 
 10  DAYS_BIRTH           36457 non-null  int64  
 11  DAYS_EMPLOYED        36457 non-null  int64  
 12  FLAG_MOBIL           36457 non-null  int64  
 13  FLAG_WORK_PHONE      36457 non-null  int64  
 14  FLAG_PHONE           36457 non-null  int64  
 15  FLAG_EMAIL           36457 non-null 

In [15]:
# Missing values
data.isna().sum().sort_values(ascending=False)


OCCUPATION_TYPE        11323
ID                         0
CODE_GENDER                0
max_month                  0
min_month                  0
months_on_book             0
any_late                   0
max_bad                    0
CNT_FAM_MEMBERS            0
FLAG_EMAIL                 0
FLAG_PHONE                 0
FLAG_WORK_PHONE            0
FLAG_MOBIL                 0
DAYS_EMPLOYED              0
DAYS_BIRTH                 0
NAME_HOUSING_TYPE          0
NAME_FAMILY_STATUS         0
NAME_EDUCATION_TYPE        0
NAME_INCOME_TYPE           0
AMT_INCOME_TOTAL           0
CNT_CHILDREN               0
FLAG_OWN_REALTY            0
FLAG_OWN_CAR               0
risk_score                 0
dtype: int64

In [16]:
# Numeric summary
data.describe()


,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,max_bad,any_late,months_on_book,min_month,max_month,risk_score
count,"36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00","36,457.00"
mean,"5,078,227.00",0.43,"186,685.74","-15,975.17","59,262.94",1.00,0.23,0.29,0.09,2.20,0.02,0.11,21.33,-26.16,-5.83,0.57
std,"41,875.24",0.74,"101,789.23","4,200.55","137,651.33",0.00,0.42,0.46,0.29,0.91,0.13,0.32,14.91,16.50,11.33,0.58
min,"5,008,804.00",0.00,"27,000.00","-25,152.00","-15,713.00",1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,-60.00,-60.00,0.02
25%,"5,042,028.00",0.00,"121,500.00","-19,438.00","-3,153.00",1.00,0.00,0.00,0.00,2.00,0.00,0.00,9.00,-39.00,-6.00,0.18
50%,"5,074,614.00",0.00,"157,500.00","-15,563.00","-1,552.00",1.00,0.00,0.00,0.00,2.00,0.00,0.00,18.00,-24.00,0.00,0.40
75%,"5,115,396.00",1.00,"225,000.00","-12,462.00",-408.00,1.00,0.00,1.00,0.00,3.00,0.00,0.00,31.00,-12.00,0.00,0.78
max,"5,150,487.00",19.00,"1,575,000.00","-7,489.00","365,243.00",1.00,1.00,1.00,1.00,20.00,1.00,1.00,61.00,0.00,0.00,4.22
